# Подсчет частотности пар товаров в продуктовых чеках

В файле содержится информация о покупках людей.

* id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
* Товар – наименование товара
* Количество – число единиц купленного товара

Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

In [1]:
%%capture
!pip install pyspark

In [2]:
import itertools
from typing import List

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import udf, col, size,explode

In [3]:
spark = SparkSession.builder.master('local[*]').appName("SparkTest").getOrCreate()

In [4]:
df = spark.read.format('csv') \
                             .options(inferSchema='true', delimiter=',', header='true') \
                             .load('/content/data_two_columns.csv')

In [5]:
df.show(5)

+---------+---------+
|invoiceno|stockcode|
+---------+---------+
|   536365|   85123A|
|   536365|    71053|
|   536365|   84406B|
|   536365|   84029G|
|   536365|   84029E|
+---------+---------+
only showing top 5 rows



In [6]:
df.printSchema()

root
 |-- invoiceno: string (nullable = true)
 |-- stockcode: string (nullable = true)



In [7]:
df_group = df.groupBy('invoiceno').agg(F.collect_set('stockcode').alias('list_stockcode')).cache()

In [8]:
df_group.show(5)

+---------+--------------------+
|invoiceno|      list_stockcode|
+---------+--------------------+
|   536596|[22900, 22114, 84...|
|   536938|[22112, 21931, 84...|
|   537252|             [22197]|
|   537691|[22505, 46000R, 2...|
|   538041|             [22145]|
+---------+--------------------+
only showing top 5 rows



In [9]:
def create_combinations_stockcode(list_stockcode:List)->List: 
 list_combinations = list(itertools.combinations(sorted(list_stockcode), 2))
 return [str(i[0])+'-'+str(i[1]) for i in list_combinations]

In [10]:
combinations_stockcode_udf = udf(lambda x: create_combinations_stockcode(x),T.ArrayType(T.StringType())) 

In [11]:
df_group_combinations = df_group.withColumn("combinations_stockcode", combinations_stockcode_udf(col("list_stockcode")))
df_group_combinations = df_group_combinations.withColumn("size_array", size(df_group_combinations.combinations_stockcode))
df_final = df_group_combinations.filter(df_group_combinations.size_array>0).select(df_group_combinations.combinations_stockcode)

In [12]:
df_final.show(7)

+----------------------+
|combinations_stockcode|
+----------------------+
|  [21624-21967, 216...|
|  [20712-20713, 207...|
|  [20975-20981, 209...|
|  [20979-21201, 209...|
|  [18098C-20676, 18...|
|  [20668-21080, 206...|
|  [21914-21915, 219...|
+----------------------+
only showing top 7 rows



In [13]:
df_final.printSchema()

root
 |-- combinations_stockcode: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [14]:
%%time
df_final.select(explode(col('combinations_stockcode')).alias("combination_stockcode")) \
        .groupBy('combination_stockcode') \
        .count() \
        .sort(col("count").desc()) \
        .show()

+---------------------+-----+
|combination_stockcode|count|
+---------------------+-----+
|         22386-85099B|  833|
|          22697-22699|  784|
|         21931-85099B|  733|
|         22411-85099B|  683|
|          20725-22383|  663|
|          20725-20727|  648|
|          22726-22727|  646|
|          22697-22698|  644|
|          22698-22699|  614|
|          20725-22384|  613|
|        85099B-85099C|  593|
|         20725-85099B|  588|
|          20727-22383|  587|
|         23203-85099B|  582|
|          20725-22382|  563|
|          20725-20728|  562|
|          22086-22910|  555|
|         23199-85099B|  555|
|          23300-23301|  549|
|          20727-22384|  547|
+---------------------+-----+
only showing top 20 rows

CPU times: user 1.09 s, sys: 121 ms, total: 1.22 s
Wall time: 3min 34s
